In [27]:
#resnet_transf.py
#Faz transfer learning usando ResNet50.
#https://medium.com/abraia/first-steps-with-transfer-learning-for-custom-image-classification-with-keras-b941601fcad5
import os; os.environ['TF_CPP_MIN_LOG_LEVEL']='3'
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'
import numpy as np;
from tensorflow import keras 
import keras.backend as K
from keras import optimizers, callbacks, regularizers
from keras.regularizers import l2
from keras.models import Sequential, Model
from keras.layers import Dropout, Conv2D, MaxPooling2D, Dense, Flatten
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions
from keras.utils import load_img, img_to_array

In [ ]:
#Descarregar segm_eliret.zip
url='http://www.lps.usp.br/hae/apostila/feiCorCrop.zip'
import os; nomeArq=os.path.split(url)[1]
if not os.path.exists(nomeArq):
  print("Baixando o arquivo",nomeArq,"para diretorio default",os.getcwd())
  os.system("wget -nc -U 'Firefox/50.0' "+url)
else:
  print("O arquivo",nomeArq,"ja existe no diretorio default",os.getcwd())
print("Descompactando arquivos novos de",nomeArq)  
os.system("unzip -u "+nomeArq) 

In [28]:
def leCsv(nomeDir, nomeArq, nl=0, nc=0):
    st=os.path.join(nomeDir,nomeArq);
    arq=open(st,"rt"); lines=arq.readlines(); arq.close(); n=len(lines)
    
    linhas_separadas=[]
    for linha in lines:
        linha=linha.strip('\n'); linha=linha.split(';'); linhas_separadas.append(linha)
    
    ay=np.empty((n),dtype='float32'); ax=np.empty((n,nl,nc,3),dtype='float32')
    for i in range(len(linhas_separadas)):
        linha=linhas_separadas[i]
        img_path=os.path.join(nomeDir,linha[0])
        t = load_img(img_path, target_size=(nl,nc))
        x = img_to_array(t)
        x = np.expand_dims(x, axis=0)
        ax[i] = preprocess_input(x)
        ay[i] = np.float32(linha[1]) #0=m ou 1=f
    return ax, ay

In [29]:
nomeprog="resnet_transf"
#Original: 280x200, redimensionado: 224x224
num_classes=2; nl=224; nc=224
batch_size = 10
diretorioBd="."
ax, ay = leCsv(diretorioBd,"treino.csv", nl=nl, nc=nc); #200 imagens
qx, qy = leCsv(diretorioBd,"teste.csv", nl=nl, nc=nc); #100 imagens
vx, vy = leCsv(diretorioBd,"valida.csv", nl=nl, nc=nc); #100 imagens
ay = keras.utils.to_categorical(ay, num_classes)
qy = keras.utils.to_categorical(qy, num_classes)
vy = keras.utils.to_categorical(vy, num_classes)

In [30]:
def modelfit(nl, nc):
    input_shape = (nl,nc,3)
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    # base_model.summary()
    x = base_model.output
    x = Flatten()(x)
    x = Dense(100, activation="relu")(x)
    predictions = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    return model

In [35]:
#def modeltrain_dense(model):
#    #Nao permite treinar base_model. So as camadas densas sao treinadas:
#    for layer in base_model.layers: layer.trainable = False
#    #Treina com learning rate grande
#    otimizador=keras.optimizers.Adam(learning_rate=1e-3)
#    model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
#    model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))

In [32]:
def modelevaluate(model,ax,ay,vx,vy,qx,qy):
    score = model.evaluate(ax, ay, verbose=0); print('Training loss:', score)
    score = model.evaluate(vx, vy, verbose=0); print('Validation loss:', score)
    score = model.evaluate(qx, qy, verbose=0); print('Test loss:', score)

In [33]:
def modeltrain_all(model,ax,ay,batch_size,vx,vy):
    #Libera todos layers do model para treinar:
    for layer in model.layers: layer.trainable = True
    #Treina com learning rate pequena todas as camadas
    otimizador=keras.optimizers.Adam(learning_rate=1e-6)
    model.compile(otimizador, loss='categorical_crossentropy', metrics =['accuracy'])
    model.fit(ax, ay, batch_size=batch_size, epochs=5, verbose=2, validation_data=(vx,vy))

In [34]:
from os import makedirs
makedirs('models')
for i in range(5):
    model = modelfit(nl,nc)
    modeltrain_all(model,ax,ay,batch_size,vx,vy)
    filename = 'models/model_' + str(i + 1) + '.h5'
    model.save(filename)
    print('>Saved %s' % filename)
    print("\n\n\n")

Epoch 1/5


KeyboardInterrupt: 

In [ ]:
from keras.models import load_model
def load_all_models(n_models):
	all_models = list()
	for i in range(n_models):
		# define filename for this ensemble
		filename = 'models/model_' + str(i + 1) + '.h5'
		# load model from file
		model = load_model(filename)
		# add to list of members
		all_models.append(model)
		print('>loaded %s' % filename)
	return all_models

In [ ]:
members = load_all_models(5)
print('Loaded %d models' % len(members))

>loaded models/model_1.h5
>loaded models/model_2.h5
>loaded models/model_3.h5
>loaded models/model_4.h5
>loaded models/model_5.h5
Loaded 5 models


In [ ]:
for model in members:
	modelevaluate(model,ax,ay,vx,vy,qx,qy)

In [ ]:
from numpy import dstack
# create stacked model input dataset as outputs from the ensemble
def stacked_dataset(members, inputX):
	stackX = None
	for model in members:
		# make prediction
		yhat = model.predict(inputX, verbose=0)
		# stack predictions into [rows, members, probabilities]
		if stackX is None:
			stackX = yhat
		else:
			stackX = dstack((stackX, yhat))
	# flatten predictions to [rows, members x probabilities]
	stackX = stackX.reshape((stackX.shape[0], stackX.shape[1]*stackX.shape[2]))
	return stackX


from sklearn.linear_model import LogisticRegression
# fit a model based on the outputs from the ensemble members
def fit_stacked_model(members, inputX, inputy):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# fit standalone model
	model = LogisticRegression()
	model.fit(stackedX, inputy)
	return model

In [ ]:
# fit stacked model using the ensemble
model = fit_stacked_model(members, qx, qy)

# make a prediction with the stacked model
def stacked_prediction(members, model, inputX):
	# create dataset using ensemble
	stackedX = stacked_dataset(members, inputX)
	# make a prediction
	yhat = model.predict(stackedX)
	return yhat

In [ ]:
from sklearn.metrics import accuracy_score
# evaluate model on test set
yhat = stacked_prediction(members, model, qx)
acc = accuracy_score(qy, yhat)
print('Stacked Test Accuracy: %.3f' % acc)